In [139]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/train.csv
/kaggle/input/coupon_item_mapping.csv
/kaggle/input/test_QyjYwdj.csv
/kaggle/input/campaign_data.csv
/kaggle/input/item_data.csv
/kaggle/input/customer_transaction_data.csv
/kaggle/input/customer_demographics.csv


In [140]:
import pandas as pd

In [141]:

train = pd.read_csv('/kaggle/input/train.csv')
camp = pd.read_csv('/kaggle/input/campaign_data.csv')
test = pd.read_csv('/kaggle/input/test_QyjYwdj.csv')
item = pd.read_csv('/kaggle/input/item_data.csv')
coup = pd.read_csv('/kaggle/input/coupon_item_mapping.csv')
cus_demo= pd.read_csv('/kaggle/input/customer_demographics.csv')
cus_trans= pd.read_csv('/kaggle/input/customer_transaction_data.csv')

In [142]:
train.shape, camp.shape, test.shape, item.shape, coup.shape, cus_demo.shape, cus_trans.shape

((78369, 5),
 (28, 4),
 (50226, 4),
 (74066, 4),
 (92663, 2),
 (760, 7),
 (1324566, 7))

In [143]:
train.isnull().sum(), camp.isnull().sum(), test.isnull().sum(), item.isnull().sum(), coup.isnull().sum(), cus_demo.isnull().sum(), cus_trans.isnull().sum()

(id                   0
 campaign_id          0
 coupon_id            0
 customer_id          0
 redemption_status    0
 dtype: int64, campaign_id      0
 campaign_type    0
 start_date       0
 end_date         0
 dtype: int64, id             0
 campaign_id    0
 coupon_id      0
 customer_id    0
 dtype: int64, item_id       0
 brand         0
 brand_type    0
 category      0
 dtype: int64, coupon_id    0
 item_id      0
 dtype: int64, customer_id         0
 age_range           0
 marital_status    329
 rented              0
 family_size         0
 no_of_children    538
 income_bracket      0
 dtype: int64, date               0
 customer_id        0
 item_id            0
 quantity           0
 selling_price      0
 other_discount     0
 coupon_discount    0
 dtype: int64)

In [144]:
cus_demo['marital_status']=cus_demo['marital_status'].fillna(method ='pad') 

In [145]:
cus_demo['no_of_children']=cus_demo['no_of_children'].fillna(method ='pad') 

In [146]:
#train=train.merge(camp, on = ['campaign_id'])
train=train.set_index("campaign_id").join(camp.set_index('campaign_id')).reset_index()
test=test.set_index("campaign_id").join(camp.set_index('campaign_id')).reset_index()
#test=test.merge(camp, on = ['campaign_id'])

In [147]:
coup=coup.merge(item, on = ['item_id'])

In [148]:
cus_trans=cus_trans.merge(item, on = ['item_id'])

In [149]:
cus_trans=cus_trans.set_index("customer_id").join(cus_demo.set_index('customer_id')).reset_index()

In [150]:
coup=coup.drop_duplicates(subset='coupon_id', keep="last")
cus_trans=cus_trans.drop_duplicates(subset='customer_id', keep="last")

In [151]:
train=train.set_index("coupon_id").join(coup.set_index('coupon_id')).reset_index()
test=test.set_index("coupon_id").join(coup.set_index('coupon_id')).reset_index()

In [152]:
#train.set_index("customer_id").join(cus_trans.set_index('customer_id')).reset_index()
train=train.merge(cus_trans, on = ['customer_id'])
test=test.merge(cus_trans, on = ['customer_id'])

In [153]:
test.shape,train.shape

((50226, 26), (78369, 27))

In [154]:
train=train.fillna(method ='bfill') 
test=test.fillna(method='bfill')

In [155]:
train=train.fillna(method ='pad')
test=test.fillna(method='pad')

In [156]:
train.dtypes

coupon_id              int64
campaign_id            int64
id                     int64
customer_id            int64
redemption_status      int64
campaign_type         object
start_date            object
end_date              object
item_id_x              int64
brand_x                int64
brand_type_x          object
category_x            object
date                  object
item_id_y              int64
quantity               int64
selling_price        float64
other_discount       float64
coupon_discount      float64
brand_y                int64
brand_type_y          object
category_y            object
age_range             object
marital_status        object
rented               float64
family_size           object
no_of_children        object
income_bracket       float64
dtype: object

In [157]:
loc = np.append(train['campaign_type'].values, test['campaign_type'].values, axis=0)
loc
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
l.fit(list(set(loc)))
train['campaign_type']=l.transform(train['campaign_type'])
test['campaign_type']=l.transform(test['campaign_type'])

In [158]:
train['start_date'] = train['start_date'].astype('datetime64')
train['end_date'] = train['end_date'].astype('datetime64')
train['date'] = train['date'].astype('datetime64')
test['start_date'] = test['start_date'].astype('datetime64')
test['end_date'] = test['end_date'].astype('datetime64')
test['date'] = test['date'].astype('datetime64')

In [159]:
loc = np.append(train['brand_type_x'].values, test['brand_type_x'].values, axis=0)
loc
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
l.fit(list(set(loc)))
train['brand_type_x']=l.transform(train['brand_type_x'])
test['brand_type_x']=l.transform(test['brand_type_x'])

In [160]:
loc = np.append(train['category_x'].values, test['category_x'].values, axis=0)
loc
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
l.fit(list(set(loc)))
train['category_x']=l.transform(train['category_x'])
test['category_x']=l.transform(test['category_x'])

In [161]:
loc = np.append(train['age_range'].values, test['age_range'].values, axis=0)
loc
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
l.fit(list(set(loc)))
train['age_range']=l.transform(train['age_range'])
test['age_range']=l.transform(test['age_range'])

In [162]:
loc = np.append(train['marital_status'].values, test['marital_status'].values, axis=0)
loc
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
l.fit(list(set(loc)))
train['marital_status']=l.transform(train['marital_status'])
test['marital_status']=l.transform(test['marital_status'])

In [163]:
loc = np.append(train['family_size'].values, test['family_size'].values, axis=0)
loc
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
l.fit(list(set(loc)))
train['family_size']=l.transform(train['family_size'])
test['family_size']=l.transform(test['family_size'])

In [164]:
loc = np.append(train['no_of_children'].values, test['no_of_children'].values, axis=0)
loc
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
l.fit(list(set(loc)))
train['no_of_children']=l.transform(train['no_of_children'])
test['no_of_children']=l.transform(test['no_of_children'])

In [165]:
loc = np.append(train['brand_type_y'].values, test['brand_type_y'].values, axis=0)
loc
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
l.fit(list(set(loc)))
train['brand_type_y']=l.transform(train['brand_type_y'])
test['brand_type_y']=l.transform(test['brand_type_y'])

In [166]:
loc = np.append(train['category_y'].values, test['category_y'].values, axis=0)
loc
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
l.fit(list(set(loc)))
train['category_y']=l.transform(train['category_y'])
test['category_y']=l.transform(test['category_y'])

In [ ]:
train_df=train
test_df=test

In [ ]:
X = train_df.drop(labels=['redemption_status'], axis=1)
y = train_df['redemption_status'].values

from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.25, random_state=1)


In [ ]:

X_train.shape, y_train.shape, X_cv.shape, y_cv.shape

In [ ]:
from math import sqrt 
from sklearn.metrics import mean_squared_log_error

In [ ]:
import lightgbm as lgb
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_cv, label=y_cv)

param = {'objective': 'regression',
         'boosting': 'gbdt',  
         'metric': 'l2_root',
         'learning_rate': 0.05, 
         'num_iterations': 1000,
         'num_leaves': 30,
         'max_depth': -1,
         'min_data_in_leaf': 4,
         'bagging_fraction': 0.78,
         'bagging_freq': 1,
         'feature_fraction': 0.65,
         }

lgbm = lgb.train(params=param,
                 verbose_eval=100,
                 early_stopping_rounds=50,
                 train_set=train_data,
                 valid_sets=[test_data])

y_pred_lgbm = lgbm.predict(X_cv)
print('RMSLE:', sqrt(mean_squared_log_error(np.expm1(y_cv), np.expm1(y_pred_lgbm))))

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(lgbm.feature_importance(), X.columns), reverse=True)[:15], 
                           columns=['Value','Feature'])
plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()

In [ ]:
Xtest = test_df

In [ ]:
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor

errlgb = []
y_pred_totlgb = []

fold = KFold(n_splits=15, shuffle=True, random_state=42)

for train_index, test_index in fold.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    param = {'objective': 'regression',
         'boosting': 'gbdt',  
         'metric': 'l2_root',
         'learning_rate': 0.05, 
         'num_iterations': 1000,
         'num_leaves': 30,
         'max_depth': -1,
         'min_data_in_leaf': 4,
         'bagging_fraction': 0.78,
         'bagging_freq': 1,
         'feature_fraction': 0.65
         }

    lgbm = LGBMRegressor(**param)
    lgbm.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0, early_stopping_rounds=50)

    y_pred_lgbm = lgbm.predict(X_test)
    print("RMSLE LGBM: ", sqrt(mean_squared_log_error(np.expm1(y_test), np.expm1(y_pred_lgbm))))

    errlgb.append(sqrt(mean_squared_log_error(np.expm1(y_test), np.expm1(y_pred_lgbm))))
    p = lgbm.predict(Xtest)
    
    y_pred_totlgb.append(p)

In [ ]:
np.mean(errlgb)

In [ ]:
lgbm_final = np.expm1(np.mean(y_pred_totlgb,0))

In [178]:
create_download_link(filename = 'sub.csv')